In [115]:
import numpy as np
import argparse
import cv2
from cnn.neural_network import CNN
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

In [116]:
# Read/Download MNIST Dataset
import cv2
import os


In [117]:
def load_images_from_folder(folders):
    images = []
    labels=[]
    print(folders)
    for folder in folders:
        print(folder)
        for filename in os.listdir(folder):
            img = cv2.imread(os.path.join(folder,filename))
            if img is not None:
                images.append(img)
                labels.append(folder)
                
    return images,labels

In [118]:
print(os.listdir)

<built-in function listdir>


In [119]:
print(os.getcwd())

D:\deep_learning_specilization\digit_recogniton_keras\Handwritten-Digit-Recognition-using-Deep-Learning-master\CNN_Keras\dataset\testing


In [120]:
import glob, os
os.chdir(r'D:\deep_learning_specilization\digit_recogniton_keras\Handwritten-Digit-Recognition-using-Deep-Learning-master\CNN_Keras\dataset\training') #change directory to where the folders are
folders = glob.glob('*')
print(folders)
train_images,train_labels=load_images_from_folder(folders)
print(len(train_images))
print(len(train_labels))
print(train_images[0].shape)
os.chdir(r'D:\deep_learning_specilization\digit_recogniton_keras\Handwritten-Digit-Recognition-using-Deep-Learning-master\CNN_Keras\dataset\testing') #change directory to where the folders are
folders = glob.glob('*')
print(folders)
test_images,test_labels=load_images_from_folder(folders)
print(len(test_images))
print(len(test_labels))
print(test_images[0].shape)


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
0
1
2
3
4
5
6
7
8
9
60000
60000
(28, 28, 3)
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
0
1
2
3
4
5
6
7
8
9
10000
10000
(28, 28, 3)


In [121]:
total_classes = 10    # 0 to 9 labels
train_labels = np_utils.to_categorical(train_labels, 10)
test_labels = np_utils.to_categorical(test_labels, 10)

In [122]:
print(len(train_images))
print(train_images[0].shape[0])

60000
28


In [123]:
x_train=np.asarray(train_images);
x_test=np.asarray(test_images);


In [124]:
print(x_train.shape)
print(x_test.shape)


(60000, 28, 28, 3)
(10000, 28, 28, 3)


In [125]:
X_train = x_train.reshape(x_train.shape[0], 28, 28, 3)
X_test = x_test.reshape(x_test.shape[0], 28, 28, 3)


In [126]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [127]:
X_train/=255
X_test/=255

In [128]:
print(train_labels[20000])

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]


In [ ]:
# Three steps to create a CNN
# 1. Convolution
# 2. Activation
# 3. Pooling
# Repeat Steps 1,2,3 for adding more hidden layers

# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

In [129]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

In [151]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

In [152]:
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [153]:
model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [154]:
model.add(Flatten())

In [155]:
# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

In [156]:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)


In [157]:
def true_pos(y_true, y_pred):
    return K.sum(y_true * K.round(y_pred))

In [158]:


def false_pos(y_true, y_pred):
    return K.sum(y_true * (1. - K.round(y_pred)))


In [159]:
def false_neg(y_true, y_pred):
    return K.sum((1. - y_true) * K.round(y_pred))

In [160]:
def precision(y_true, y_pred):
    return true_pos(y_true, y_pred) /((true_pos(y_true, y_pred) + false_pos(y_true, y_pred)))

In [161]:
def recall(y_true, y_pred):
    return true_pos(y_true, y_pred) /( (true_pos(y_true, y_pred) + false_neg(y_true, y_pred)))

In [162]:
def f1_score(y_true, y_pred):
    return 2. / (1. / recall(y_true, y_pred) + 1. / precision(y_true, y_pred))

In [163]:
#metrics=['accuracy',mean_pred]

In [164]:
#for fn in [precision, recall, f1_score]:
 #   metrics.append(fn)

In [165]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy',precision,recall,f1_score])

In [166]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)

test_gen = ImageDataGenerator()

In [167]:
train_generator = gen.flow(X_train, train_labels, batch_size=64)
test_generator = test_gen.flow(X_test, test_labels, batch_size=64)

In [169]:
model.fit_generator(train_generator, steps_per_epoch=60000//64, epochs=1, 
                    validation_data=test_generator, validation_steps=10000//64)

Epoch 1/1
937/937 [==============================] - 429s 458ms/step - loss: 0.1126 - acc: 0.9644 - precision: 0.9592 - recall: 0.9717 - f1_score: 0.9654 - val_loss: 0.0323 - val_acc: 0.9898 - val_precision: 0.9888 - val_recall: 0.9913 - val_f1_score: 0.9900


In [170]:
from keras.models import load_model

model.save('D:\deep_learning_specilization\digit_recogniton_keras\Handwritten-Digit-Recognition-using-Deep-Learning-master\CNN_Keras\my_modelepoch1.h5') 



In [171]:
model = load_model('D:\deep_learning_specilization\digit_recogniton_keras\Handwritten-Digit-Recognition-using-Deep-Learning-master\CNN_Keras\my_modelepoch1.h5')

ValueError: Unknown metric function:precision

In [172]:
model.save_weights('D:\deep_learning_specilization\digit_recogniton_keras\Handwritten-Digit-Recognition-using-Deep-Learning-master\CNN_Keras\my_model_weight.h5')

In [174]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 26, 26, 32)        896       
_________________________________________________________________
batch_normalization_17 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
activation_61 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_18 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
activation_62 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 12, 12, 32)        0         
__________

In [175]:
score = model.evaluate(X_test, test_labels, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0322425770608
Test accuracy: 0.9898
